In [35]:
#WARNING THIS SCRIPT TAKES A LONG TIME TO RUN!
#Note Everythin is in natural units so C = 1
import sys, os
if __package__ is None:
    import sys, os
    sys.path.append(os.path.realpath("/data/shared/Software/"))
    sys.path.append(os.path.realpath("../../"))
from CMS_Deep_Learning.utils.data_parse import ROOT_to_pandas
from CMS_Deep_Learning.utils.data_parse import DataProcessingProcedure
from CMS_Deep_Learning.utils.data_parse import leaves_from_obj
import ROOT
from ROOT import TTree
import numpy as np
import pandas as pd
import ntpath
import glob
from itertools import cycle, islice

#didit = False

In [36]:
def cullNonObservables(frame):
    #Status of 1 means that the particle is a stable product
    stable_cond = frame["Status"] == 1 
    #All even leptons are neutrinos which we can't measure
    notNeutrino_cond = frame["PID"] % 2 == 1
    parton_cond = np.abs(frame["PID"]) <= 8
    #Get all entries that satisfy the conditions
    frame = frame[stable_cond & notNeutrino_cond]
    #Drop the Status frame since we only needed it to see if the particle was stable
    frame = frame.drop(["Status"], axis=1)
    return frame

#Define the speed of light C
#Turns C=1 since everything is in natural units :/
#C = np.float64(2.99792458e8); #m/s
mass_of_electron = np.float64(0.0005109989461) #eV/c
mass_of_muon = np.float64(0.1056583715)      #eV/c
#def four_vec_func(inputs):
#        E = inputs[0]
#        Eta = inputs[1]
#        Phi = inputs[2]
#        PT = inputs[3]
#        E_over_c = E/C
#        px = E_over_c * np.sin(Phi) * np.cos(Eta) 
#        py = E_over_c * np.sin(Phi) * np.sin(Eta)
#        pz = E_over_c * np.cos(Phi)
#        print(np.sqrt(px*px + py*py), PT/C)
#        return [E_over_c, px, py, pz]

#def getPandasPhotons(filename):
#    four_vec_inputs, dummy = leaves_from_obj("Photon", ["PT", "Eta", "Phi"])
#    four_vec_proc = DataProcessingProcedure(lambda x: four_vec_from_PT(x,0), four_vec_inputs, ["E/c", "Px","Py","Pz"])

#    PID_proc = DataProcessingProcedure(lambda x:[22], [], ["PID"])
#    charge_proc = DataProcessingProcedure(lambda x:[0], [], ["Charge"])

#    columns=[four_vec_proc, PID_proc, charge_proc]
#    leaves, columns = leaves_from_obj("Photon", columns+["PT", "Eta", "Phi"])

#    photon_frame = ROOT_to_pandas(filename,
#                          leaves,
#                          columns=columns,
#                          verbosity=1)
#    return photon_frame


extra_data = ["PT", "Phi", "Eta"]


def four_vec_from_PT(inputs, M):
    #print(type(inputs[0]))
    PT = inputs[0] #Units ?
    Eta = inputs[1]
    Phi = inputs[2]
    #M has units of eV/(c^2)
    if(M == None):
        M = inputs[3]
    momentum_mag = PT * np.cosh(Eta)
    #if(~didit):
    #print(momentum_mag, PT, np.cosh(Eta))
    #    didit = True
    E_over_c = np.sqrt(np.square(M) + np.square(momentum_mag))
    #print(PT/np.sin(Phi))
    px = PT * np.cos(Phi) 
    py = PT * np.sin(Phi) 
    pz = PT * np.sinh(Eta)
    return [E_over_c, px, py, pz]

def getPandasParticles(filename, cull=True):
    #C=1 in natural units so no processing needs to be done
    E_over_c_proc = DataProcessingProcedure(lambda x:x[0], ["Particle.E"], ["E/c"])
    columns= [E_over_c_proc, "Px", "Py", "Pz", "PID", "Charge", "Status"]
    leaves, columns = leaves_from_obj("Particle", columns+extra_data)
    original_frame = ROOT_to_pandas(filename,
                                 leaves,
                                  columns=columns,
                                  verbosity=1)
    if(cull):
        particle_frame = cullNonObservables(original_frame)
    else:
        particle_frame = original_frame
    return particle_frame


def getPandasSpecificParticles(filename, name, mass=None, pid=None, chrg_def=-1, charge=None):
    if(mass != None):
        four_vec_inputs, dummy = leaves_from_obj(name, ["PT", "Eta", "Phi"])
    else:
        four_vec_inputs, dummy = leaves_from_obj(name, ["PT", "Eta", "Phi", "Mass"])
        
    four_vec_proc = DataProcessingProcedure(lambda x: four_vec_from_PT(x,mass)
                                            , four_vec_inputs, ["E/c", "Px","Py","Pz"])
    
    status_proc = DataProcessingProcedure(lambda x:[1], [], ["Status"])
    
    if(charge == None):
        if(pid != None):
            PID_charge_proc = DataProcessingProcedure(lambda x: [pid*chrg_def*x[0], x[0]]
                                                      , [name + ".Charge"], ["PID", "Charge"])
            columns=[four_vec_proc, PID_charge_proc, status_proc]
        else:
            columns=[four_vec_proc, "PID", "Charge", status_proc]
    else:
        charge_proc = DataProcessingProcedure(lambda x: [charge], [], ["Charge"])
        if(pid != None):
            PID_proc = DataProcessingProcedure(lambda x: [pid], [], ["PID"])
            columns=[four_vec_proc, PID_proc, charge_proc,status_proc]
        else:
            columns=[four_vec_proc, "PID", charge_proc,status_proc]
        
        
        
        
    
    leaves, columns = leaves_from_obj(name, columns+extra_data)

    #Extract the tables from the root file
    frame = ROOT_to_pandas(filename,
                          leaves,
                          columns=columns,
                          verbosity=1)
    return frame

def getPandasPhotons(filename):
    return getPandasSpecificParticles(filename, "Photon", mass=0, pid=22, charge=0)

def getPandasElectrons(filename):
    return getPandasSpecificParticles(filename, "Electron", mass=mass_of_electron, pid=11)

def getPandasTightMuons(filename):
    return getPandasSpecificParticles(filename, "MuonTight", mass=mass_of_muon, pid=13)

def getPandasJets(filename):
    return getPandasSpecificParticles(filename, "Jet", pid=100, chrg_def=1)

def getPandasMissingET(filename, name="MissingET"):  
    pid = 83
    if(name == "PuppiMissingET"): pid=84
    four_vec_inputs, dummy = leaves_from_obj(name, ["MET", "Eta", "Phi"])
    four_vec_proc = DataProcessingProcedure(lambda x: four_vec_from_PT(x,0)
                                            , four_vec_inputs, ["E/c", "Px","Py","Pz"])
    status_proc = DataProcessingProcedure(lambda x:[1], [], ["Status"])
    charge_proc = DataProcessingProcedure(lambda x: [0], [], ["Charge"])
    PID_proc = DataProcessingProcedure(lambda x: [pid], [], ["PID"])
    columns=[four_vec_proc, PID_proc, charge_proc, status_proc]
    met_proc = DataProcessingProcedure(lambda x: [x[0]], ["MissingET.MET"], ["PT"] )
    
    ex = [x if (x != "PT") else met_proc for x in extra_data]
    leaves, columns = leaves_from_obj(name, columns+ex)

    #Extract the tables from the root file
    frame = ROOT_to_pandas(filename,
                          leaves,
                          columns=columns,
                          verbosity=1)
    return frame
        
        


def getPandasAll(filename, cull=False, includePuppi=True):
    lst = [getPandasPhotons(filename),
                    getPandasElectrons(filename),
                    getPandasTightMuons(filename),
                    getPandasJets(filename),
                    getPandasParticles(filename, cull=cull),
                    getPandasMissingET(filename)]
    
    if(includePuppi):
        lst = lst + [getPandasMissingET(filename, "PuppiMissingET")]
    #Merge all these frames
    out = pd.concat(lst)
    return out

#def getPandasAll(filename):
#    out = pd.concat([getPandasPhotons(filename),getPandasParticles(filename, cull=False)])
#    return out

#http://stackoverflow.com/questions/3678869/pythonic-way-to-combine-two-lists-in-an-alternating-fashion
def roundrobin(*iterables):
    "roundrobin('ABC', 'D', 'EF') --> A D E B F C"
    # Recipe credited to George Sakkis
    pending = len(iterables)
    nexts = cycle(iter(it).next for it in iterables)
    while pending:
        try:
            for next in nexts:
                yield next()
        except StopIteration:
            pending -= 1
            nexts = cycle(islice(nexts, pending))
    
#def groupEntriesToArrays(frame, select):
#    out = []
#    m = frame['Entry'].max()
#    for entry in range(0, m+1):
#        cond = frame['Entry'] == entry
#        entry_frame = frame[cond]
#        entry_frame = entry_frame[select]
#        arr = np.array(entry_frame)
#        np.random.shuffle(arr)
#        out.append(arr)
#    return out

#def groupEntriesToArrays(frame, select):
#    grouped = frame.groupby(["Entry"]).apply(lambda df: df[select].tolist())
    #print(grouped)
#    return grouped

In [37]:
#frame = getPandasPhotons("/data/shared/Delphes/ttbar_13TeV/ttbar_13TeV_80.root")
#print(frame)
#cond = np.abs(frame["PID"]) == 13
#frame_cond = frame[cond]
#print(frame_cond)
#frame = getPandasSpecificParticles("/data/shared/Delphes/ttbar_13TeV/ttbar_13TeV_80.root", "Photon", mass=0, pid=22, charge = 0)
#print(frame)

#frame = getPandasParticles("/data/shared/Delphes/ttbar_13TeV/ttbar_13TeV_80.root")
#cond = np.abs(frame["PID"]) == 11
#frame_cond = frame[cond]
#print(frame_cond)

#print(sys.float_info)
#frame = getPandasElectrons("/data/shared/Delphes/ttbar_13TeV/ttbar_13TeV_80.root")
#print(frame)
#frame = getPandasSpecificParticles("/data/shared/Delphes/ttbar_13TeV/ttbar_13TeV_80.root", "Electron", mass=mass_of_electron, pid=11)
#print(frame)

In [38]:
#frame = getPandasParticles("/data/shared/Delphes/ttbar_13TeV/ttbar_13TeV_80.root")
#cond = np.abs(frame["PID"]) == 11
#frame_cond = frame[cond]
#print(frame_cond)

#frame = getPandasAll("/data/shared/Delphes/ttbar_13TeV/ttbar_13TeV_80.root")
#print(frame)

#frame = getPandasMissingET("/data/shared/Delphes/ttbar_13TeV/ttbar_13TeV_80.root", name="PuppiMissingET")
#print(frame)

In [39]:
#print(frame)
#cond = np.abs(frame["PID"]) == 84
#frame_cond = frame[cond]
#print(frame_cond)

In [ ]:
ttbar_files = glob.glob("/data/shared/Delphes/ttbar_13TeV/*.root")
WJet_files = glob.glob("/data/shared/Delphes/WJets_lepFilter_13TeV/*.root")
WJet_files = glob.glob("/data/shared/Delphes/qcd_lepFilter_13TeV/*.root")



ttbar_out_dir = "/data/shared/Delphes/ttbar_13TeV/parsed_pandas/"
if not os.path.exists(ttbar_out_dir):
    os.makedirs(ttbar_out_dir)
    
WJet_out_dir = "/data/shared/Delphes/WJets_13TeV/parsed_pandas/"
if not os.path.exists(WJet_out_dir):
    os.makedirs(WJet_out_dir)
    
QCD_out_dir = "/data/shared/Delphes/WJets_13TeV/parsed_pandas/"
if not os.path.exists(QCD_out_dir):
    os.makedirs(QCD_out_dir)

ttbar_jobs = []
WJet_jobs = []
QCD_jobs = []
    
for ttbar_file in ttbar_files:
    ttbar_filename = os.path.splitext(ntpath.basename(ttbar_file))[0]
    ttbar_jobs.append((ttbar_file, ttbar_out_dir + ttbar_filename + ".h5"))

for WJet_file in WJet_files:
    WJet_filename = os.path.splitext(ntpath.basename(WJet_file))[0]
    WJet_jobs.append((WJet_file, WJet_out_dir + WJet_filename + ".h5"))
    
#TODO: Add QCD
    
jobs = roundrobin(ttbar_jobs, WJet_jobs)

In [ ]:
for job in jobs:
    frame = getPandasAll(job[0])
    frame.to_hdf(job[1], 'data', mode='w')

Extracting data from /data/shared/Delphes/ttbar_13TeV/ttbar_13TeV_80.root
Using trees: Delphes
Procedure at column 1 maps "['Photon.PT', 'Photon.Eta', 'Photon.Phi'] -> ['E/c', 'Px', 'Py', 'Pz']"
Procedure at column 5 maps "[] -> ['PID']"
Procedure at column 6 maps "[] -> ['Charge']"
Procedure at column 7 maps "[] -> ['Status']"
Extracting Leaves: Photon.PT, Photon.Eta, Photon.Phi, Photon.PT, Photon.Phi, Photon.Eta
Renaming to: E/c, Px, Py, Pz, PID, Charge, Status, PT, Phi, Eta
[=================   ] 8917/10000       Entry         E/c         Px         Py          Pz    ...     Charge  \
0          0   15.772708  -3.238023 -13.761521    6.993857    ...          0   
1          1   34.376894  21.462844   3.886852   26.570840    ...          0   
2          1   18.028188  11.152195   2.201617   13.992747    ...          0   
3          2   14.662694 -11.851376   5.636919   -6.539465    ...          0   
4          2   22.323748  10.945124   4.119174  -19.015425    ...          0   
...  